In [37]:
from Assignment3_part1 import *
from tqdm import tqdm
from scipy.stats import norm

## Variables qui nous intéressent

* SAA_weights_IS
* SAA_weights_OS
* TAA_weights_IS
* TAA_weights_OS
* weight_target_IS
* weight_target_OS
* benchmark_weights_IS
* benchmark_weights_OS

### Benchmark (50% Equities / 50% Bond)

### Return

* returns_IS_price
* returns_OS_price

In [19]:
# SAA
SAA_active_weight_IS = SAA_weights_IS - benchmark_weights_IS
SAA_active_weight_OS = SAA_weights_OS - benchmark_weights_OS

# TAA
TAA_active_weight_IS = TAA_weights_IS - benchmark_weights_IS
TAA_active_weight_OS = TAA_weights_OS - benchmark_weights_OS

# Real portfolio
active_weight_target_IS = weight_target_IS - benchmark_weights_IS
active_weight_target_OS = weight_target_OS - benchmark_weights_OS

# Question 1

In [31]:
MCR_SAA_active_IS = MCR_calc(SAA_active_weight_IS,returns_IS_price)

MCR_TAA_active_IS = MCR_calc(TAA_active_weight_IS,returns_IS_price)

MCR_real_IS = MCR_calc(active_weight_target_IS,returns_IS_price)

In [87]:
# SAA
MCR_SAA = []
MCR_SAA_final = []
MCR_TAA = []
MCR_TAA_final = []
MCR_real = []
MCR_real_final = []

SAA_rolling = SAA_active_weight_IS.copy()
TAA_rolling = TAA_active_weight_IS.copy()
Real_rolling = weight_target_IS.copy()
return_rolling = returns_IS_price.copy()
for i in tqdm(range(len(SAA_active_weight_OS))):
    SAA_rolling = SAA_rolling.append(SAA_active_weight_OS.iloc[i,:])
    TAA_rolling = TAA_rolling.append(TAA_active_weight_OS.iloc[i,:])
    Real_rolling = Real_rolling.append(active_weight_target_OS.iloc[i,:])

    return_rolling = return_rolling.append(returns_OS_price.iloc[i,:])
    MCR_SAA.append(MCR_calc(SAA_rolling,return_rolling))
    MCR_TAA.append(MCR_calc(TAA_rolling,return_rolling))
    MCR_real.append(MCR_calc(Real_rolling,return_rolling))
    #print(MCR_SAA[-1])
    MCR_SAA_final.append(MCR_SAA[-1].iloc[:,-1])
    MCR_TAA_final.append(MCR_TAA[-1].iloc[:,-1])
    MCR_real_final.append(MCR_real[-1].iloc[:,-1])
    
SAA_MCR = pd.DataFrame(MCR_SAA_final[:])
TAA_MCR = pd.DataFrame(MCR_TAA_final[:])
real_MCR = pd.DataFrame(MCR_real_final[:])

100%|██████████| 124/124 [00:04<00:00, 29.55it/s]


In [86]:
SAA_MCR = pd.DataFrame(MCR_SAA_final[:])
SAA_MCR

,0,1,2,3,4,5,6
2011-01-31,-0.049001,0.003644,-0.002335,-0.020048,0.000907,-0.020199,-0.040097
2011-02-28,-0.048889,0.003645,-0.002330,-0.019966,0.000674,-0.020027,-0.039949
2011-03-31,-0.048703,0.003623,-0.002327,-0.019887,0.000673,-0.019851,-0.039844
2011-04-29,-0.048626,0.003587,-0.002370,-0.019817,0.000301,-0.020036,-0.039558
2011-05-31,-0.048485,0.003592,-0.002321,-0.019724,0.000164,-0.020245,-0.039507
...,...,...,...,...,...,...,...
2020-12-31,-0.044592,0.003438,-0.002794,-0.017455,0.002136,-0.029934,-0.032105
2021-01-29,-0.044503,0.003428,-0.002793,-0.017419,0.002122,-0.029854,-0.032039
2021-02-26,-0.044412,0.003497,-0.002687,-0.017341,0.002390,-0.030325,-0.032278
2021-03-31,-0.044352,0.003503,-0.002614,-0.017266,0.002459,-0.030123,-0.032075


# Question 2

## Variance Covariance method

## VaR

In [45]:
ptf_return_SAA = np.multiply(SAA_active_weight_OS,returns_OS_price).sum(axis=1)
mean_SAA = np.mean(ptf_return_SAA)
std_SAA = np.std(ptf_return_SAA)

ptf_return_TAA = np.multiply(TAA_active_weight_OS,returns_OS_price).sum(axis=1)
mean_TAA = np.mean(ptf_return_TAA)
std_TAA = np.std(ptf_return_TAA)

ptf_return_Real = np.multiply(active_weight_target_OS,returns_OS_price).sum(axis=1)
mean_Real = np.mean(ptf_return_Real)
std_Real = np.std(ptf_return_Real)

In [47]:
VaR_95_SAA = norm.ppf(1-0.95,mean_SAA,std_SAA)
print(f'VaR 95% SAA: {VaR_95_SAA}')

VaR_95_TAA = norm.ppf(1-0.95,mean_TAA,std_TAA)
print(f'VaR 95% TAA: {VaR_95_TAA}')

VaR_95_Real = norm.ppf(1-0.95,mean_Real,std_Real)
print(f'VaR 95% Real: {VaR_95_Real}')

VaR 95% SAA: -0.03292615618768459
VaR 95% TAA: -0.06868917746927439
VaR 95% Real: -0.06523311983250694


## Historical method

### VaR

In [54]:
ptf_return_SAA_sort = ptf_return_SAA.sort_values(ascending=True)
ptf_return_TAA_sort = ptf_return_TAA.sort_values(ascending=True)
ptf_return_Real_sort = ptf_return_Real.sort_values(ascending=True)


print(f'VaR SAA 5%: {ptf_return_SAA_sort.quantile(0.05)}')
print(f'VaR TAA 5%: {ptf_return_TAA_sort.quantile(0.05)}')
print(f'VaR Real 5%: {ptf_return_Real_sort.quantile(0.05)}')

VaR SAA 5%: -0.027072011521389647
VaR TAA 5%: -0.05694802610459951
VaR Real 5%: -0.056638707509634775


In [93]:
#https://s3.amazonaws.com/assets.datacamp.com/production/course_5612/slides/chapter4.pdf

ES_SAA_hist = ptf_return_SAA_sort[ptf_return_SAA_sort <= np.quantile(ptf_return_SAA_sort,0.05)].mean()
ES_TAA_hist = ptf_return_TAA_sort[ptf_return_TAA_sort <= np.quantile(ptf_return_TAA_sort,0.05)].mean()
ES_Real_hist = ptf_return_Real_sort[ptf_return_Real_sort <= np.quantile(ptf_return_Real_sort,0.05)].mean()

print(f'ES SAA hist 5%: {ES_SAA_hist}')
print(f'ES TAA hist 5%: {ES_TAA_hist}')
print(f'ES Real hist 5%: {ES_Real_hist}')

ES SAA hist 5%: -0.03757842294207015
ES TAA hist 5%: -0.07091502938402984
ES Real hist 5%: -0.06817750539266482
